In [1]:
import pandas as pd
import numpy as np

import prepare as prp

import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nickolaspedrimiranda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nickolaspedrimiranda/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
repo = pd.read_csv('all_repos.csv')

In [3]:
repo

,repo,language,readme_contents
0,ScottfreeLLC/AlphaPy,Python,AlphaPy\n=======\n\n|badge_pypi| |badge_downlo...
1,jalapic/engsoccerdata,R,"Latest GitHub version: 11/4/2022, v0.1.7\n\nN..."
2,bttmly/nba,JavaScript,# nba\n*Node.js client for nba.com API endpoin...
3,kyleskom/NBA-Machine-Learning-Sports-Betting,Python,# NBA Sports Betting Using Machine Learning 🏀\...
4,linouk23/NBA-Player-Movements,Python,# NBA Player Movements\n\nThis is a script for...
...,...,...,...
792,abanoubragheb1/EventProject,C#,# EventProject\nA project that helps those int...
793,hemvall/Heracles,Vue,# Heracles\nSports Performance Tracker: A REST...
794,Chew/MLB-GameFeed-Bot,Java,# MLB-GameFeed-Bot\n\nAdd to your server with ...
795,paucls/court-booking-ddd-kotlin,Kotlin,# court-booking-ddd\nA sample application usin...


In [4]:
repo = prp.cleanse(repo, 'readme_contents')

In [5]:
valid_languages = ["Python", 'HTML', "JavaScript", "R", "Java", "TypeScript", "PHP", "Ruby", "C#", "C++", "Dart", "Kotlin", "Objective-C", "Swift", "Go", "Rust", "C", "Elixir", "CoffeeScript", "MATLAB", "Visual Basic .NET", "Scala", "Haskell", "Stata", "Haxe", "Lua", "Perl", "Clojure"]

In [6]:
len(repo)

797

In [7]:
repo.language.value_counts()

Python               232
JavaScript           141
R                     43
TypeScript            40
HTML                  40
Java                  39
PHP                   35
Ruby                  30
C#                    24
C++                   22
Dart                  22
Kotlin                20
CSS                   19
Vue                   10
Objective-C           10
Swift                 10
Go                     9
Rust                   6
XSLT                   4
TeX                    3
Shell                  3
Solidity               3
C                      3
Elixir                 3
CoffeeScript           2
MATLAB                 2
Visual Basic .NET      2
Svelte                 2
EJS                    2
Matlab                 2
Scala                  2
Haskell                1
Stata                  1
Visual Basic           1
Haxe                   1
Dockerfile             1
XML                    1
Max                    1
Batchfile              1
q                      1


In [8]:
langauges = repo.language.value_counts().index

In [9]:
not_lang = list(set(langauges).difference(set(valid_languages)))

In [10]:
repo = pd.read_csv('all_repos.csv')
repo = prp.cleanse(repo, 'readme_contents')
repo.language = prp.top_languages(repo, 4)
repo.head()

,repo,language,readme_contents,clean,stemmed,lemmatized
0,ScottfreeLLC/AlphaPy,Python,AlphaPy\n=======\n\n|badge_pypi| |badge_downlo...,alphapy badgepypi badgedownloads badgedocs bad...,alphapi badgepypi badgedownload badgedoc badge...,alphapy badgepypi badgedownloads badgedocs bad...
1,jalapic/engsoccerdata,R,"Latest GitHub version: 11/4/2022, v0.1.7\n\nN...",latest github version 1142022 v017 nov 2022 up...,latest github version 1142022 v017 nov 2022 up...,latest github version 1142022 v017 nov 2022 up...
2,bttmly/nba,JavaScript,# nba\n*Node.js client for nba.com API endpoin...,nba nodejs client nbacom api endpoints npm ins...,nba nodej client nbacom api endpoint npm insta...,nba nodejs client nbacom api endpoint npm inst...
3,kyleskom/NBA-Machine-Learning-Sports-Betting,Python,# NBA Sports Betting Using Machine Learning 🏀\...,nba sports betting using machine learning img ...,nba sport bet use machin learn img srchttpsgit...,nba sport betting using machine learning img s...
4,linouk23/NBA-Player-Movements,Python,# NBA Player Movements\n\nThis is a script for...,nba player movements script visualization nba ...,nba player movement script visual nba game raw...,nba player movement script visualization nba g...


In [11]:
repo, val, test = prp.train_val_test(repo, 'language', stratify=True)

(557, 6) (120, 6) (120, 6)


In [12]:
['all'] + list(repo.language.unique())

['all', 'Other', 'R', 'Python', 'JavaScript', 'TypeScript']

In [13]:
repo.language.value_counts()

Other         238
Python        162
JavaScript     99
R              30
TypeScript     28
Name: language, dtype: int64

In [14]:
def word_counts(dataframe, col, data):
    df = dataframe.copy()
    uniques = df[col].unique()
    col_names = ['all'] + list(df[col].unique())
    
    word_counts = []
    
    all_words = df[data]
    all_words = pd.Series(' '.join(all_words).split()).value_counts()
    word_counts.append(all_words)
    
    for name in uniques:
        words = df[data][df[col] == name]
        words = pd.Series(' '.join(words).split()).value_counts()
        word_counts.append(words)
    
    word_counts = pd.concat(word_counts, axis=1)\
    .set_axis(col_names, axis=1).fillna(0)\
    .apply(lambda s: s.astype(int))
    
    return word_counts

In [15]:
all_words = word_counts(repo, 'language', 'clean')
word_filter = list(all_words[all_words['all'] < 10].index)
word_filter[:10]

['gambling',
 'override',
 'managers',
 'begins',
 'accessed',
 'slack',
 'batch',
 'coding',
 'overlap',
 'careful']

In [16]:
python_words = repo.clean[repo.language == 'Python']
python_words = pd.Series(' '.join(python_words).split()).value_counts()

javascript_words = repo.clean[repo.language == 'JavaScript']
javascript_words = pd.Series(' '.join(javascript_words).split()).value_counts()

r_words = repo.clean[repo.language == 'R']
r_words = pd.Series(' '.join(r_words).split()).value_counts()

typescript_words = repo.clean[repo.language == 'TypeScript']
typescript_words = pd.Series(' '.join(typescript_words).split()).value_counts()

other_words = repo.clean[repo.language == 'Other']
other_words = pd.Series(' '.join(other_words).split()).value_counts()


all_words = repo.clean
all_words = pd.Series(' '.join(all_words).split()).value_counts()

In [17]:
python_words

barry          758
data           461
file           376
python         343
vanessa        291
              ... 
services         1
multitude        1
'elevation'      1
tidying          1
v30              1
Length: 14320, dtype: int64

In [18]:
word_counts = pd.concat([all_words, python_words, javascript_words, r_words, typescript_words, other_words], axis=1)\
.set_axis(['all', 'python', 'javascript', 'r', 'typescript', 'other'], axis=1).fillna(0)\
.apply(lambda s: s.astype(int))

In [19]:
word_counts

,all,python,javascript,r,typescript,other
data,1105,461,109,144,115,276
barry,808,758,50,0,0,0
1,743,241,133,71,21,277
sports,702,225,136,33,37,271
use,687,243,94,42,58,250
...,...,...,...,...,...,...
impacted,1,0,0,0,0,1
1841699,1,0,0,0,0,1
1908364,1,0,0,0,0,1
meanscore,1,0,0,0,0,1


In [23]:
unique_words = list(word_counts.index)

In [32]:
total_appearances = []

for word in unique_words:
    n = 0
    for row in repo.clean:
        words = set(row.split())
        if word in words:
            n += 1
    total_appearances.append(n)
    

In [27]:
word_counts['total_appearances'] = total_appearances

In [36]:
word_counts.sort_values(by='total_appearances', ascending=False)[:30]

,all,python,javascript,r,typescript,other,total_appearances
sports,702,225,136,33,37,271,301
use,687,243,94,42,58,250,258
data,1105,461,109,144,115,276,229
run,645,232,100,6,74,233,225
install,599,283,92,29,21,174,219
using,493,185,59,34,46,169,211
1,743,241,133,71,21,277,193
project,487,125,93,14,43,212,188
file,634,376,68,20,30,140,185
2,512,157,88,36,18,213,182
